In [34]:
import os
import pandas as pd

In [35]:
def read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE):

    files = [os.path.join(DIR_INPUT, f) for f in os.listdir(DIR_INPUT) if f>=BEGIN_DATE+'.pkl' and f<=END_DATE+'.pkl']

    frames = []
    for f in files:
        df = pd.read_pickle(f)
        frames.append(df)
        del df
    df_final = pd.concat(frames)

    df_final=df_final.sort_values('TRANSACTION_ID')
    df_final.reset_index(drop=True,inplace=True)
    #  Note: -1 are missing values for real world data
    df_final=df_final.replace([-1],0)

    return df_final


In [36]:
DIR_INPUT='./simulated-data-raw/'
BEGIN_DATE = "2018-04-01"
END_DATE = "2018-09-30"

read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))



/tmp/ipykernel_6160/3006431160.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final=df_final.replace([-1],0)
/tmp/ipykernel_6160/3006431160.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final=df_final.replace([-1],0)


CPU times: user 3.97 s, sys: 302 ms, total: 4.27 s
Wall time: 4.28 s
1754155 transactions loaded, containing 14763 fraudulent transactions


In [37]:
transactions_df.head()


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,1997,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,4448,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1338,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,3730,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,2705,50.99,634,0,0,0


In [38]:
def is_weekend(tx_datatime):

    weekday = tx_datatime.weekday()
    is_weekend = weekday >= 5

    return int(is_weekend)


In [39]:
%time transactions_df['TX_DURING_WEEKEND'] = transactions_df.TX_DATETIME.apply(is_weekend)

print(transactions_df['TX_DURING_WEEKEND'].value_counts())

CPU times: user 4.63 s, sys: 163 ms, total: 4.79 s
Wall time: 4.8 s
TX_DURING_WEEKEND
0    1245770
1     508385
Name: count, dtype: int64


In [40]:
def is_night(tx_datetime):

    tx_hour = tx_datetime.hour
    is_night = tx_hour<=6

    return int(is_night)

In [41]:
%time transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

CPU times: user 4.5 s, sys: 172 ms, total: 4.67 s
Wall time: 4.81 s


In [42]:
transactions_df[transactions_df.TX_TIME_DAYS>=30]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT
288062,288062,2018-05-01 00:01:21,3546,376,18.71,2592081,30,0,0,0,1
288063,288063,2018-05-01 00:01:48,206,3768,18.60,2592108,30,0,0,0,1
288064,288064,2018-05-01 00:02:22,2610,1156,66.67,2592142,30,0,0,0,1
288065,288065,2018-05-01 00:03:15,4578,1560,79.41,2592195,30,0,0,0,1
288066,288066,2018-05-01 00:03:51,1246,5923,52.08,2592231,30,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,7152,54.24,15810996,182,0,0,1,0
1754151,1754151,2018-09-30 23:57:38,4342,5486,1.23,15811058,182,0,0,1,0
1754152,1754152,2018-09-30 23:58:21,618,3727,6.62,15811101,182,0,0,1,0
1754153,1754153,2018-09-30 23:59:52,4056,1039,55.40,15811192,182,0,0,1,0


In [ ]:
def get_customer_spending_behaviour_features(customer_transactions, window_size_in_days=[1,7,30]):
    customer_transactions = customer_transactions.sort_values('TX_DATETIME')

    customer_transactions.index = customer_transactions.TX_DATETIME

    for window_size in window_size_in_days:


In [64]:
c0.to_csv("b.csv")

In [63]:
c0 = transactions_df[transactions_df.CUSTOMER_ID==0].sort_values('TX_DATETIME')
c0.index = c0.TX_DATETIME

sum_amount_tx = c0['TX_AMOUNT'].rolling(str(3)+'d').sum()
sum_amount_tx.to_csv("a.csv")